In [6]:
import re
import itertools
from pathlib import Path
import scipy
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from typing import Any, Dict, List, Tuple, Optional

In [3]:
fpath = Path('../data/2975_LickingLama_20250207_125807.mat')

In [5]:
save_path =  r"E:\GLM_project\EncoderResults"
Path(save_path).mkdir(parents=True, exist_ok=True)

In [7]:
opts = dict(
        skipIfExist=False,
        showOrthplot=False,
        preTrigDur=0.5,
        postTrigDur=8.0,
        nrFolds=20,
        testFrac=0.1,
        removeAutoTrials=True,
        innateTask=False,
        videoDims=100,
        preStimDur=0.2,
        postStimDur=3.0,
        preMoveDur=0.2,
        postMoveDur=3.0,
        cvRegs=["time", "choice", "prevChoice", "success", "prevSuccess", "water", "licks", "video"],
    )



In [8]:
import h5py

vc_path = Path('../data/Vc_2975.mat')
print(f"Loading variables from {vc_path}")

Loading variables from ..\data\Vc_2975.mat


In [ ]:
with h5py.File(vc_path, 'r') as f:
    # Load the specific variables
    Vc = f['Vc'][:]
    trialOn = f['trialOn'][:]
    U = f['U'][:]
    ledInfo = f['ledInfo'][:] 
    frameInfo = f['frameInfo'][:]  
    
    print(f"Vc shape: {Vc.shape}")
    print(f"trialOn shape: {trialOn.shape}")
    print(f"U shape: {U.shape}")
    print(f"ledInfo shape: {ledInfo.shape}")
    print(f"frameInfo shape: {frameInfo.shape}") 

# Vc shape: (number of frames, number of components) 41623x500
# U shape: (number of pixels, number of components) 540x640x500
# frameInfo shape: (number of frames, time) 41623x2

Vc shape: (41623, 500)
trialOn shape: (3, 1000000)
U shape: (500, 640, 540)
ledInfo shape: (3, 41623)
frameInfo shape: (2, 41623)


In [ ]:
trialOn_clean = trialOn[:, ~np.isnan(trialOn[0, :])]
trialOn_clean = trialOn_clean.T
trialOn_clean.shape

# trialOn_clean is now (number of valid trials, 3)
#  where the first column is frame number, 
# second column is trial start times and 
# the third column is trail number 

(274, 3)

In [19]:
nrTrials = trialOn_clean.shape[0]
nrTrials

274

In [31]:
trialOnFrames = np.zeros(nrTrials, dtype=int)  # like zeros(1, nrTrials)

for i in range(nrTrials):
    # MATLAB: frameInfo(:,2) - trialOn_clean(i,2)
    diff = frameInfo[:, 1] - trialOn_clean[i, 1]

    idxs = np.where(diff > 0)[0]
    if idxs.size == 0:
        # MATLAB would return [] here; choose a sentinel:
        trialOnFrames[i] = 0      # 0 means "not found" (since MATLAB indices start at 1)
    else:
        trialOnFrames[i] = idxs[0] + 1  # +1 to mimic MATLAB indexing

In [32]:
trialOnFrames

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])